In [12]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType

import os

In [13]:
# change to be reflective of your environment
data_dir = '/Users/zhaoluyang/Downloads/Senior-Capstone-2018-2019-master/Notebooks/TestData'

In [14]:
import os
size = os.path.getsize(data_dir+"/BACKUP_OBJECTS.csv")
end = 50 # how many gigs to scale backup objects
upscale = int((end * 1073741824) / size)
upscale

13

In [15]:
# initiates a SparkContext which is necessary for accessing data in Spark
sc = SparkContext()
sqlContext = SQLContext(sc)
# change to match your environment
output_dir = data_dir + "/merge_data"

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by getOrCreate at <ipython-input-1-a5c6d49f12a3>:138 

In [ ]:
SDRO = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir + '/SD_RECON_ORDER.csv'])
SS_POOLS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir + '/SS_POOLS.csv'])
AFBF = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/AF_BITFILES.csv"])
BACKUP_OBJECTS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/BACKUP_OBJECTS.csv"])
ls = [SDRO, SS_POOLS, AFBF, BACKUP_OBJECTS]

In [ ]:
# upscale step
for i in range(len(ls)):
    csv = ls[i]
    concat = csv
    print(concat.count())
    for j in range(upscale):
        concat = concat.union(csv)
    ls[i] = concat
    print(concat.count())
    
    
SDRO = ls[0]
SS_POOLS = ls[1]
AFBF = ls[2]
BACKUP_OBJECTS = ls[3]

# # save new ones before join then reload
# SDRO.write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/upscale_" + str(end) + "_SDRO")
# SS_POOLS.write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/upscale_" + str(end) + "_SS_POOLS")
# AFBF.write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/upscale_" + str(end) + "_AFBF")
# BACKUP_OBJECTS.write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/upscale_" + str(end) + "_BACKUP_OBJECTS")


In [ ]:
# SDRO = None
# SS_POOLS = None
# AFBF = None
# BACKUP_OBJECTS = None

In [ ]:
# SDRO = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load(data_dir + "/upscale_" + str(end) + "_SDRO/*.csv")
# SS_POOLS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load(data_dir + "/upscale_" + str(end) + "_SS_POOLS/*.csv")
# AFBF = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load(data_dir + "/upscale_" + str(end) + "_AFBF/*.csv")
# BACKUP_OBJECTS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load(data_dir + "/upscale_" + str(end) + "_BACKUP_OBJECTS/*.csv")

In [ ]:
full_outer_join = BACKUP_OBJECTS.join(AFBF, BACKUP_OBJECTS.OBJID == AFBF.BFID,how='left') # Could also use 'full_outer'
full_outer_join = full_outer_join.join(SDRO, ['OBJID'],how='left') # Could also use 'full_outer'

In [ ]:
SDRO = None
SS_POOLS = None
AFBF = None
BACKUP_OBJECTS = None

In [ ]:
df = full_outer_join
df = df.filter(df.POOLID. isNotNull())
df = df.withColumn("POOLID", df["POOLID"].cast("int"))
df = df.filter(df.ATTRLENGTH. isNotNull())
df = df.withColumn("ATTRLENGTH", df["ATTRLENGTH"].cast("float"))
df = df.filter(df.BFSIZE. isNotNull())
df = df.withColumn("BFSIZE", df["BFSIZE"].cast("float"))
df = df.filter(df.HDRSIZE. isNotNull())
df = df.withColumn("HDRSIZE", df["HDRSIZE"].cast("float"))
df = df.filter(df.OBJID. isNotNull())
df = df.withColumn("OBJID", df["OBJID"].cast("float"))

In [ ]:
from pyspark.sql.functions import when   

df = df.withColumn('POOLID', when(df['POOLID'] == -1000000, 0).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == -9, 1).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 4, 2).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 6, 3).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 42, 4).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 72, 5).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 82, 6).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == -1, 7).otherwise(df['POOLID']))

In [ ]:
df = df.filter(df.POOLID. isNotNull())
df = df.withColumn("POOLID", df["POOLID"].cast("int"))
df = df.filter(df.ATTRLENGTH. isNotNull())
df = df.withColumn("ATTRLENGTH", df["ATTRLENGTH"].cast("float"))
df = df.filter(df.BFSIZE. isNotNull())
df = df.withColumn("BFSIZE", df["BFSIZE"].cast("float"))
df = df.filter(df.HDRSIZE. isNotNull())
df = df.withColumn("HDRSIZE", df["HDRSIZE"].cast("float"))
df = df.filter(df.OBJID. isNotNull())
df = df.withColumn("OBJID", df["OBJID"].cast("float"))

In [ ]:
df.select("ATTRLENGTH", "BFSIZE", "HDRSIZE", "POOLID").write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/merge_data/upscale")
                                                                                                    
                                                                                                                                                   
                                                                                                    

In [ ]:
df = None

In [ ]:
sc.stop()

In [44]:
# Dependencies
import glob, os
import tensorflow as tf
import pandas as pd
import numpy as np

# create seed for random_normal()
seed = 1234
np.random.seed(seed)
tf.set_random_seed(seed)
input_nodes = 5
output_nodes = 3
hidden_layer_nodes = 10

In [45]:
# Data preprocess
def data_preprocess(next):
    tf_features = [next['BFSIZE'], next['NODESTATE'], next['METADATASIZE'], next['STG_HINT'], next['BFSIZE']]
    # tf_features = [next['BFSIZE'], next['HDRSIZE'], next['METADATASIZE']]
    features_columns = np.array(sess.run(tf_features), dtype='float32')

    num_features = len(tf_features)
    # Normalization
    for column in range(num_features):
        column_range = features_columns[column].max() - features_columns[column].min()
        # print(column_range)
        if column_range != 0.0:
            features_columns[column] = (features_columns[column] - features_columns[column].min()) / column_range

    features = np.array([]).reshape(0, num_features)
    for i in range(batch_len):
        ele = np.zeros(num_features)
        for j in range(num_features):
            np.put(ele, j, features_columns[j][i])
        features = np.r_[features, [ele]]
    # print(len(features))

    tf_labels = next['label']
    labels = np.array(sess.run(tf_labels), dtype='float32')
    # print(labels)

    # One-hot encoding for the categories
    num_classes = 3
    targets = np.array([]).reshape(0, num_classes)

    for i in range(0, batch_len):
        ele = np.zeros(num_classes)
        np.put(ele, labels[i], 1)
        targets = np.r_[targets, [ele]]

    #     return features, targets

    #     Shuffle Data
    indices = np.random.choice(len(features), len(features), replace=False)
    X_data = features[indices]
    y_data = targets[indices]

    # X_values = features[indices]
    # y_values = targets[indices]

    return X_data, y_data


In [46]:
def training(X_train, y_train, start_epoch):
    # Interval / Epochs
    interval = 500
    epoch = 1500

    # Training the model...
    for i in range(1, (epoch + 1)):
        sess.run(optimizer, feed_dict={X_data: X_train, y_target: y_train})
        if i % interval == 0:
            print('Epoch', i + start_epoch, '|', 'Loss:',
                  sess.run(loss, feed_dict={X_data: X_train, y_target: y_train}))
            # print(sess.run(b1))

    return i + start_epoch


In [47]:
# get the accuracy of the model
def predict(X_test, y_test):
    correct_prediction = tf.equal(tf.argmax(final_output, 1), tf.argmax(y_target,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("The accuracy of the model is: ", sess.run(accuracy, feed_dict={X_data: X_test, y_target: y_test}))


In [48]:
# define a neural network

# Initialize placeholders
X_data = tf.placeholder(shape=[None, input_nodes], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, output_nodes], dtype=tf.float32)

# We create a neural Network which contains 3 layers with 4, 10, 5 nodes repectively
w1 = tf.Variable(tf.random_normal(shape=[input_nodes, hidden_layer_nodes]))  # Weight of the input layer
b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes]))  # Bias of the input layer
w2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes, output_nodes]))  # Weight of the hidden layer
b2 = tf.Variable(tf.random_normal(shape=[output_nodes]))  # Bias of the hidden layer
hidden_output = tf.nn.relu(tf.add(tf.matmul(X_data, w1), b1))
final_output = tf.nn.softmax(tf.add(tf.matmul(hidden_output, w2), b2))

# Loss Function
loss = tf.reduce_mean(-tf.reduce_sum(y_target * tf.log(final_output), axis=0))

# Optimizer
# optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(loss)
# optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate=0.001).minimize(loss)
# optimizer = tf.train.ProximalAdagradOptimizer(learning_rate=0.01).minimize(loss)
# optimizer = tf.train.FtrlOptimizer(learning_rate=0.01).minimize(loss)
optimizer = tf.train.AdagradOptimizer(learning_rate=0.1).minimize(loss)
# optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
# optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.01).minimize(loss)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

print("A neural Network which contains 3 layers with", input_nodes, ", ", hidden_layer_nodes, ", ", output_nodes,
      " nodes repectively was created!")

A neural Network which contains 3 layers with 5 ,  10 ,  3  nodes repectively was created!


In [49]:
path = '/Users/zhaoluyang/Downloads/Senior-Capstone-2018-2019-master/Notebooks/firstTestData/merge_data/feature_extraction'
# path = data_dir + "/merge_data/upscale"
all_files = glob.glob(
    os.path.join(path, "*.csv"))  # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_dataset = pd.concat(df_from_each_file, ignore_index=True)
# 44633877
# print(len(concatenated_dataset))

batch_len = 4000
dataset = tf.data.experimental.make_csv_dataset(all_files, batch_size=batch_len)
# print(dataset)

start_epoch = 0
print('Training the model...')
iter = dataset.make_one_shot_iterator()

# Initialize variables
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# There are total 2,157,764 rows in the combined csv files

# choose first 4000 rows as test data
next = iter.get_next()

X_test, y_test = data_preprocess(next)

# then 538*4000 = 2,152,000 rows will be used as training data
# 538
for i in range(20):
    next = iter.get_next()
    # next is a dict with key=columns names and value=column data
    X_train, y_train = data_preprocess(next)
    #     print(X_train)
    start_epoch = training(X_train, y_train, start_epoch)

predict(X_test, y_test)

print("Training finished\n")

Training the model...
Epoch 500 | Loss: 686.616
Epoch 1000 | Loss: 686.0283
Epoch 1500 | Loss: 685.7827
Epoch 2000 | Loss: 692.50946
Epoch 2500 | Loss: 692.18463
Epoch 3000 | Loss: 692.01764
Epoch 3500 | Loss: 685.9758
Epoch 4000 | Loss: 685.7102
Epoch 4500 | Loss: 685.53534
Epoch 5000 | Loss: 630.41766
Epoch 5500 | Loss: 630.2012
Epoch 6000 | Loss: 630.06146
Epoch 6500 | Loss: 687.795
Epoch 7000 | Loss: 687.66797
Epoch 7500 | Loss: 687.5865
Epoch 8000 | Loss: 680.63336
Epoch 8500 | Loss: 680.1218
Epoch 9000 | Loss: 679.8581
Epoch 9500 | Loss: 690.70074
Epoch 10000 | Loss: 690.49896
Epoch 10500 | Loss: 690.33716
Epoch 11000 | Loss: 620.8725
Epoch 11500 | Loss: 620.4439
Epoch 12000 | Loss: 620.29083
Epoch 12500 | Loss: 691.8057
Epoch 13000 | Loss: 691.4412
Epoch 13500 | Loss: 691.22845
Epoch 14000 | Loss: 679.8381
Epoch 14500 | Loss: 679.5461
Epoch 15000 | Loss: 679.3776
Epoch 15500 | Loss: 672.19635
Epoch 16000 | Loss: 671.8801
Epoch 16500 | Loss: 671.79877
Epoch 17000 | Loss: 659.3924

In [ ]:
# Prediction
np.set_printoptions(precision=4)
unknown = np.array([[0.0, 0.002894, 0.148097]], dtype=np.float32)
predicted = sess.run(final_output, feed_dict={X_data: unknown})
# model.predict(unknown)
print("Using model to predict pool id for features: ", unknown)
print("\nPredicted softmax vector is: ",predicted)
Class_dict={'POOLID_0': 0, 'POOLID_1': 1, 'POOLID_2': 2}
pool_dict = {v:k for k,v in Class_dict.items()}
print("\nPredicted pool id is: ", pool_dict[np.argmax(predicted)])


Tensor("strided_slice_1:0", shape=(), dtype=float32)
